In [1]:
import plotly.graph_objects as go
import networkx as nx
import os

# reading input log file

# test_name = "ddmd"
# test_name = "vist"
# test_name = "s9f9p8"
# test_name = "f24s9p12_1"
# test_name = "ss_f24s9p12" # mapping to workload_name=pyflextrkr
# test_name = "f48s9p24_1" # mapping to workload_name=pyflextrkr
# test_name = "h5bench-sync-write-3d-contig-contig-read-full-80g_3"

# stat_path=f"example_stat/{test_name}"
# image_path=f"{stat_path}/images"

# test_name = "strong_scale"
WORKLOAD_NAME = "h5bench" # choices: pyflextrkr --> f48s9p24, h5bench --> 

VFD_ACCESS_SKIP=10

ADD_ADDR=False


In [2]:
# My utility functions
import utils.stat_loader as sload
import utils.stat_print as sp
import utils.vfd_stat2graph as vfd2g
import utils.vfd_graph2sankey as vfd2sk
import utils.full_stat2graph as f2g

# Improve Functions
## TODO
- Add dataset nodes
- Add file address ordering nodes


In [3]:
def show_detail_overhead_old(stat_type, file_dict, task_list, workflow_time=0, workload_name="pyflextrkr"):
    overhead_dict = {
        "VOL": ["VOL-Init(us)", "VOL-Term(us)", "VOL-Log(us)",
                "VOL-HT-Add(us)", "VOL-HT-Rm(us)", "VOL-HT-Search(us)",
                "VOL-FILE_LL-Add(us)", "VOL-FILE_LL-Rm(us)", "VOL-FILE_LL-Update(us)",
                "VOL-DS_LL-Add(us)", "VOL-DS_LL-Rm(us)", "VOL-DS_LL-Update(us)",
                "VOL-GRP_LL-Add(us)", "VOL-GRP_LL-Rm(us)",
                "VOL-DT_LL-Add(us)", "VOL-DT_LL-Rm(us)"],

        "VFD": ["VFD-Init(us)", "VFD-Term(us)", "VFD-Tracker-Init(us)", "VFD-Stat-Add(us)", 
                "VFD-Stat-Update(us)", "VFD-Stat-Rm(us)", "VFD-Stat-Log(us)"]
    }
    
    
    overhead_time_list = []
    first_file_time = -1
    last_file_time = -1
    
    file_overhead_dict = {}
    
    # initialize the dict with task_list and overhead_type
    for pid_file, pid_stat in file_dict.items():
        file_overhead_dict[pid_file] = {}
        for task in task_list:
            file_overhead_dict[pid_file][task] = {}
            for overhead_type in overhead_dict[stat_type]:
                file_overhead_dict[pid_file][task][overhead_type] = []

    for pid_file, pid_stat in file_dict.items():
        prev_task = None
        task_pid = (pid_file.split("/")[-1]).split("-")[0]      
        for entry in pid_stat:
            if "Task" in entry.keys():
                task_time = entry["Task"]
                overhead_time_list.append(task_time)
                if stat_type == "VOL":
                    
                    if prev_task is not None:
                        for overhead_type in overhead_dict[stat_type]:
                            file_overhead_dict[pid_file][prev_task][overhead_type].append(task_time[overhead_type])
                
                elif stat_type == "VFD":
                    task_name = task_time['task_name']
                    # task_name = task_name.split("-")[0]
                    task_name = task_name.replace(f"-{task_pid}","")
                    for overhead_type in overhead_dict[stat_type]:
                        file_overhead_dict[pid_file][task_name][overhead_type].append(task_time[overhead_type])
                    # Get the other entry.keys() to get first and last task time, ther eis only 2 keys 
                    other_keys = [key for key in entry.keys() if key != "Task"]
                    other_key = other_keys[0]
                    first_file_time, last_file_time = get_first_last_file_time(entry[other_key], first_file_time, last_file_time)
            else:
                
                file_id = list(entry.keys())[0]
                first_file_time, last_file_time = get_first_last_file_time(entry[file_id], first_file_time, last_file_time)
                
                if "task_name" in entry[file_id].keys():
                    task_name = entry[file_id]['task_name']
                    # split task_name-pid to get task_name only
                    # task_name = task_name.split("-")[0]
                    task_name = task_name.replace(f"-{task_pid}","")
                    prev_task = task_name
    
     
    if workload_name == "pyflextrkr":
        task_overhead_dict = {}
        # initialize the dict with task_list and overhead_type
        for task in task_list:
            task_overhead_dict[task] = {}
            for overhead_type in overhead_dict[stat_type]:
                task_overhead_dict[task][overhead_type] = 0

        for pid_file, overhead_time_dict in file_overhead_dict.items():
            # print(f"{pid_file}: {overhead_time_dict}")
            for task in task_list:
                for overhead_type in overhead_time_dict[task]:
                    pid_overhead = sum(overhead_time_dict[task][overhead_type])
                    task_overhead = task_overhead_dict[task][overhead_type]
                    
                    # # pick only 
                    if pid_overhead > task_overhead:
                        task_overhead_dict[task][overhead_type] = pid_overhead

        # show task_overhead_dict
        for task in task_list:
            print(f"{task}: {task_overhead_dict[task]}")
    
        # Sum all overhead except VOL-Overhead(us)/VFD-Overhead(us)
        for task in task_list:
            task_overhead_dict[task]["Sum-Overhead(us)"] = 0
            for overhead_type in overhead_dict[stat_type]:
                # Pick the largest value of overhead_type
                # print(f'{task}: {overhead_type}: {overhead_time_dict[task][overhead_type]}')
                task_overhead_dict[task]["Sum-Overhead(us)"] += task_overhead_dict[task][overhead_type]
            print(f'{task}: Sum-Overhead(us): {task_overhead_dict[task]["Sum-Overhead(us)"]}')

        if workflow_time == 0:
            workflow_time = last_file_time - first_file_time # get a estimate of workflow time from first file to last file time
        print(f"Total task time (=workflow time) (us): {workflow_time}")
        
        # Sum all tasks overhead
        sum_tasks_overhead = 0
        for task in task_list: sum_tasks_overhead += task_overhead_dict[task]["Sum-Overhead(us)"]
            
    elif workload_name == "h5bench":
        sum_tasks_overhead = 0
        for pid_file, overhead_time_dict in file_overhead_dict.items():
            # print(f"{pid_file}: {overhead_time_dict}")
            for task in task_list:
                for overhead_type in overhead_time_dict[task]:
                    pid_overhead = sum(overhead_time_dict[task][overhead_type])
                    # task_overhead = task_overhead_dict[task][overhead_type]
                    sum_tasks_overhead += pid_overhead

    # Show overhead percentage on total task time
    print(f"Sum tasks overhead (us): {sum_tasks_overhead}")
    print(f"Workflow time (us): {workflow_time}")
    print(f"Overhead percentage on total task time: {sum_tasks_overhead/workflow_time*100}%")
    
    return file_overhead_dict, sum_tasks_overhead

# Search for log file lines to get workflow time
def get_workflow_time(stat_path:str, workload_name:str=WORKLOAD_NAME):
    # Find any file with ".log" extension
    log_file = None
    for file in os.listdir(stat_path):
        if file.endswith(".log"):
            log_file = os.path.join(stat_path, file)
            break
    print(f"log_file = {log_file}")

    WORKFLOW_TIME = 0
    # Read file and search for workflow time starting with
    if workload_name == "pyflextrkr":
        # TIME_STR_START = "Pyflextrkr TIME:"
        TIME_STR_START = "Execution time:"
        with open(log_file, "r") as f:
            for line in f:
                if TIME_STR_START in line:
                    WORKFLOW_TIME = float(line.split(" ")[2])
                    # Convert from ms to us
                    WORKFLOW_TIME = WORKFLOW_TIME * 1000
                    break
    elif workload_name == "h5bench": 
        TIME_STR_START = "Execution time"
        with open(log_file, "r") as f:
            for line in f:
                if TIME_STR_START in line:
                    runtime = float(line.split(" ")[3])
                    print(f"runtime = {runtime}")
                    # Convert from ms to us
                    WORKFLOW_TIME += runtime * 1000
    else:
        print("WORKLOAD_NAME not recognized")
        WORKFLOW_TIME = 0

    print(f"get_workflow_time() WORKFLOW_TIME = {WORKFLOW_TIME} us")
    return WORKFLOW_TIME


# WORKFLOW_TIME = get_workflow_time(stat_path)
# print(f"WORKFLOW_TIME = {WORKFLOW_TIME} us")

In [4]:
# function to get the workflow time, which is the sum of the write and read times in each workloads
def get_workflow_times(write_times, read_times):
    workflow_times = {}
    for ds_size in write_times.keys():
        workflow_times[f'{ds_size}'] = [w + r for w,r in zip(write_times[ds_size], read_times[ds_size])]
    return workflow_times

def get_workflow_time_from_log(log_file, workload_name="pyflextrkr"):
    workflow_time = 0
    if workload_name == "pyflextrkr":
        TIME_STR_START = "Pyflextrkr TIME:"
        with open(log_file, "r") as f:
            for line in f:
                if TIME_STR_START in line:
                    workflow_time = float(line.split(" ")[2])
                    # Convert from ms to microseconds
                    workflow_time = workflow_time * 1000
                    break
    elif workload_name == "h5bench": 
        TIME_STR_START = "Execution time"
        with open(log_file, "r") as f:
            for line in f:
                if TIME_STR_START in line:
                    runtime = float(line.split(" ")[3])
                    print(f"runtime = {runtime}")
                    # Convert from ms to microseconds
                    workflow_time += runtime * 1000
    else:
        print("WORKLOAD_NAME not recognized")

    return workflow_time

def get_avg_workflow_time(dataset_prefix_name, rcnt=0, workload_name="h5bench", proc_num=1):
    trial_nums = 3
    workflow_times = []
    processed_folder = []
    
    # for i in list(["t1", "t2", "t3"]):
    #     stat_path = f"{dataset_prefix_name}_{i}"
    #     processed_folder.append(stat_path)
    #     workflow_time = get_workflow_time(stat_path, workload_name=workload_name)
    #     print(f"{stat_path} workflow_time: {workflow_time}")
    #     workflow_times.append(workflow_time)
    for i in list(["t1", "t2", "t3"]):
        stat_path = f"{dataset_prefix_name}_{i}"
        processed_folder.append(stat_path)
        max_workflow_time = 0
        if rcnt > 0:
            file_name = f"{stat_path}/sync-write-3d-contig-contig-read-{rcnt}r-h5bench.log"
            workflow_t = get_workflow_time_from_log(file_name, workload_name=workload_name)
            max_workflow_time += workflow_t
            workflow_times.append(workflow_t)
        else:
            file_name = f"{stat_path}/sync-write-3d-contig-contig-read-2r-h5bench.log"
            workflow_t = get_workflow_time_from_log(file_name, workload_name=workload_name)
            max_workflow_time += workflow_t
            workflow_times.append(workflow_t)

        workflow_time = max_workflow_time
    
    avg_workflow = sum(workflow_times)/trial_nums
    # print(f"processed_folder: {processed_folder}")
    # print(f"avg_workflow: {avg_workflow}")
    return avg_workflow

def get_avg_vfd_vol_overhead(dataset_prefix_name, TASK_LISTS, workload_n="pyflextrkr"):
    trial_nums = 3
    vfd_overheads = []
    vol_overheads = []
    print(f"Running workload: {workload_n}")
    for i in list(["t1", "t2", "t3"]):
        base_stat_path = f"{dataset_prefix_name}_{i}"
        vfd_files = sload.find_files_with_pattern(base_stat_path, "vfd") 
        print(f"vfd_files: {vfd_files}")
        vfd_dict = sload.load_stat_json(vfd_files)
        print("loading vfd json done")
        vol_files = sload.find_files_with_pattern(base_stat_path, "vol") 
        print(f"vol_files: {vol_files}")
        vol_dict = sload.load_stat_json(vol_files)
        print("loading vol json done")
        print("Handle VFD and VOL overhead")
        workflow_t = get_workflow_time(base_stat_path, workload_name=workload_n)
        print(f"get_avg_vfd_vol_overhead() workflow_t: {workflow_t}")
        _, sum_vfd_overhead = show_detail_overhead_old("VFD", vfd_dict, TASK_LISTS, workflow_time=workflow_t, workload_name=workload_n) #
        _, sum_vol_overhead = show_detail_overhead_old("VOL", vol_dict, TASK_LISTS, workflow_time=workflow_t, workload_name=workload_n) #
        vfd_overheads.append(sum_vfd_overhead)
        vol_overheads.append(sum_vol_overhead)
    
    avg_vfd_overhead = sum(vfd_overheads)/trial_nums
    avg_vol_overhead = sum(vol_overheads)/trial_nums
    return avg_vfd_overhead, avg_vol_overhead

# functions to get the vfd and vol overheads (%) via increasing the scalce
def get_h5bench_strong_scale_overheads(base_path, test_name, dt_size, ds_num, read_cnts=[]):
    overheads = {}
    overheads_times = {}
    for dt in dt_size:
        for size in ds_num:
            if read_cnts:
                for rcnt in read_cnts:
                    # for trial in list(["t1", "t2", "t3"]):
                    stat_path = f"{base_path}/{test_name}/sync-write-3d-contig-contig-read-{rcnt}r"
                    #  get the average workflow time of the dataset
                    avg_workflow_time = get_avg_workflow_time(stat_path, rcnt=rcnt, workload_name="h5bench")
                    print(f"avg_workflow_time (us): {avg_workflow_time}")
                    #  get the avergae overhead time of the dataset
                    # TASK_LISTS = ["h5_write", "h5_read"]
                    TASK_LISTS = [f"sync-write-3d-contig-contig-read-{rcnt}r"]
                    avg_vfd_overhead, avg_vol_overhead = get_avg_vfd_vol_overhead(stat_path, TASK_LISTS, workload_n="h5bench")

                    overheads[f'{dt}Arr-{rcnt}R'] = {'vfd_overhead(%)': (avg_vfd_overhead/avg_workflow_time*100), 'vol_overhead(%)': avg_vol_overhead/avg_workflow_time*100}
                    overheads_times[f'{dt}D-{rcnt}R'] = {'vfd_overhead(us)': avg_vfd_overhead, 'vol_overhead(us)': avg_vol_overhead, 'workflow_time(us)': avg_workflow_time}
            else:

                # for trial in list(["t1", "t2", "t3"]):
                stat_path = f"{base_path}/{test_name}/sync-write-3d-contig-contig-read"
                #  get the average workflow time of the dataset
                avg_workflow_time = get_avg_workflow_time(stat_path, workload_name="pyflextrkr")
                print(f"avg_workflow_time (us): {avg_workflow_time}")
                #  get the avergae overhead time of the dataset
                # TASK_LISTS = ["h5_write", "h5_read"]
                TASK_LISTS = [f"sync-write-3d-contig-contig-read-{rcnt}r"]
                avg_vfd_overhead, avg_vol_overhead = get_avg_vfd_vol_overhead(stat_path, TASK_LISTS, workload_n="pyflextrkr")

                overheads[f'{dt}Arr-{size}DS'] = {'vfd_overhead(%)': (avg_vfd_overhead/avg_workflow_time*100), 'vol_overhead(%)': avg_vol_overhead/avg_workflow_time*100}
                overheads_times[f'{dt}D-{size}G'] = {'vfd_overhead(us)': avg_vfd_overhead, 'vol_overhead(us)': avg_vol_overhead, 'workflow_time(us)': avg_workflow_time}

    return overheads, overheads_times

def get_first_last_file_time(entry, first_file_time, last_file_time):
    if "open_time(us)" in entry.keys():
        task_open_time = entry['open_time(us)']
        if first_file_time == -1 or task_open_time < first_file_time:
            first_file_time = task_open_time
    if "close_time(us)" in entry.keys():
        task_close_time = entry['close_time(us)']
        if last_file_time == -1 or task_close_time > last_file_time:
            last_file_time = task_close_time
    return first_file_time, last_file_time

In [5]:
# H5BENCH_DT_PATH=f"/Users/yejie/Projects/SCS_Lab/SC24/Candice_Paper/candice_data/h5bench"
H5BENCH_DT_PATH="/Users/mengtang/script/dayu-tracker/flow_analysis/h5bench"
subtest_name="nread_overhead" # ssd3d64_200ds_reads ssd3d64_rerun1 ssd3d64_200ds_reads
arr_size=["5g"]
# ds_num=[2, 20, 200, 2000, 20000] # 20000
ds_num=[8] # 20000

read_cnts=[1, 2, 4, 8, 16, 32, 64, 256, 1024]

# get the overheads of the strong scale
strong_overheads, strong_verheads_times = get_h5bench_strong_scale_overheads(H5BENCH_DT_PATH, subtest_name, arr_size, ds_num, read_cnts)


# ToDo: Plot the overheads of the strong scale
# H5BENCH_DT_PATH="/Users/mengtang/script/dayu-tracker/flow_analysis/example_stat/strong_scale"

runtime = 21416.0
runtime = 21383.0
runtime = 22369.0
avg_workflow_time (us): 21722666.666666668
Running workload: h5bench
vfd_files: ['/Users/mengtang/script/dayu-tracker/flow_analysis/h5bench/nread_overhead/sync-write-3d-contig-contig-read-1r_t1/249318-vfd_data_stat.json', '/Users/mengtang/script/dayu-tracker/flow_analysis/h5bench/nread_overhead/sync-write-3d-contig-contig-read-1r_t1/249321-vfd_data_stat.json']
loading /Users/mengtang/script/dayu-tracker/flow_analysis/h5bench/nread_overhead/sync-write-3d-contig-contig-read-1r_t1/249318-vfd_data_stat.json
loading /Users/mengtang/script/dayu-tracker/flow_analysis/h5bench/nread_overhead/sync-write-3d-contig-contig-read-1r_t1/249321-vfd_data_stat.json
loading vfd json done
vol_files: ['/Users/mengtang/script/dayu-tracker/flow_analysis/h5bench/nread_overhead/sync-write-3d-contig-contig-read-1r_t1/249318-vol_data_stat.json', '/Users/mengtang/script/dayu-tracker/flow_analysis/h5bench/nread_overhead/sync-write-3d-contig-contig-read-1r_t1/249

In [6]:
print("*************************************************************************************************************")
print("Strong Scale Overheads")
# print(strong_overheads)
for k,v in strong_overheads.items():
    print(f"{k}: {v}")

print()

print("Strong Scale Overheads Times")
# print(strong_verheads_times)
for k,v in strong_verheads_times.items():
    print(f"{k}: {v}")


*************************************************************************************************************
Strong Scale Overheads
5gArr-1R: {'vfd_overhead(%)': 0.09721483857107781, 'vol_overhead(%)': 0.04590596611834029}
5gArr-2R: {'vfd_overhead(%)': 0.10174539668460607, 'vol_overhead(%)': 0.05311582303515564}
5gArr-4R: {'vfd_overhead(%)': 0.10164078564300501, 'vol_overhead(%)': 0.06079867382537052}
5gArr-8R: {'vfd_overhead(%)': 0.10343486355214536, 'vol_overhead(%)': 0.06709026389339585}
5gArr-16R: {'vfd_overhead(%)': 0.11718003976453159, 'vol_overhead(%)': 0.06586457867889392}
5gArr-32R: {'vfd_overhead(%)': 0.10679163920816132, 'vol_overhead(%)': 0.07834299226639273}
5gArr-64R: {'vfd_overhead(%)': 0.09611463980651042, 'vol_overhead(%)': 0.06850302599534694}
5gArr-256R: {'vfd_overhead(%)': 0.10122800191697072, 'vol_overhead(%)': 0.07230619542288624}
5gArr-1024R: {'vfd_overhead(%)': 0.09963540145034762, 'vol_overhead(%)': 0.07344081739458147}

Strong Scale Overheads Times
5gD-1R: {'